# Mode solver Fully Vectorial

```
    _________________________________

                                    clad_height
           wg_width
         <---------->
          ___________    _ _ _ _ _ _
         |           |
    _____|           |____          |
                                    wg_heigth
    slab_height                     |
    _______________________ _ _ _ _ __

    sub_height
    _________________________________
    <------------------------------->
                 sub_width

```


In [ ]:
import modesolverpy as ms

In [ ]:
s = ms.mode_solver_full(plot=True, n_modes=1, wg_width=0.5, wg_height=0.22, fields_to_write=('Ex'), overwrite=True)

In [ ]:
s = ms.mode_solver_full(plot=True) # by default it will plot at the fields

In [ ]:
ms.mode_solver_full?

where the waveguide parameters 

In [ ]:
s.results['n_effs'][0].real

Lets do some convergence tests

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import modesolverpy as ms

dx=np.linspace(0.01, 0.02, 3)
n_effs = [ms.mode_solver_full(n_modes=1, x_step=dxi, y_step=dxi, plot=False).results['n_effs'][0].real for dxi in dx]

In [ ]:
plt.plot(dx, n_effs)